In [1]:
#!pip install bs4==0.0.1 #if running in virtual notebook 

In [2]:
#Import the libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import re

In [3]:
#Get the webpage and parse it as html

page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")
soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
#Retrieve the day names nested in body 

day_names = soup.find_all('h3', class_ = 'DailyContent--daypartName--3G5Y8') #day names are in this tag

list_of_days = []
for each_day in day_names:
    e = each_day.text                #'.text' returns the text inside the Tag
    list_of_days.append(e)

list_of_days = list_of_days[0:10]    #only getting the first 10 days details
#print(list_of_days)

#First pattern for the DATE
date_string = ' '.join(list_of_days)  
print(date_string)
date = re.findall("[A-Z][a-z]+ [0-9]+", date_string) #use regex to filter the date 

#Second pattern for Day and Night. 
first_day = list_of_days[0]           #check for first day and create a list based on that
day_checker =re.findall(" [A-Z][a-z]+",first_day)
print(day_checker)

if ' Night' in day_checker:
    type_of_day = ['Night','Day']*5   
elif ' Day' in day_checker:
    type_of_day = ['Day','Night']*5   #Day and Night is a repetitive pattern, so creating it is easy rather than extracting it 
print(type_of_day)

[' Day']
['Day', 'Night', 'Day', 'Night', 'Day', 'Night', 'Day', 'Night', 'Day', 'Night']


In [13]:
#Retrieve the weather summary nested inside the body and find temperatures

day_temp = soup.find_all('div', class_ = 'DetailsSummary--temperature--3FMlw') 

temp_list = []
for summary in day_temp:
    e = summary.text              
    temp_list.append(e)

#Extract the temperature by searching for a regex pattern and filter it further
#print(temp_list)
temp_string = ' '.join(temp_list)            
temp = re.findall("([0-9]\d[°])", temp_string)      
temp = ' '.join(temp).replace('°',' ').split()     
print(temp)


['60°/50°', '62°/54°', '64°/54°', '63°/55°', '63°/52°', '65°/52°', '67°/52°', '69°/52°', '67°/51°', '64°/51°', '65°/50°', '66°/51°', '67°/51°', '66°/51°', '67°/52°']
['60', '50', '62', '54', '64', '54', '63', '55', '63', '52', '65', '52', '67', '52', '69', '52', '67', '51', '64', '51', '65', '50', '66', '51', '67', '51', '66', '51', '67', '52']


In [12]:
#Convert Fahrenheit to celcius 

temp_celcius = []
def celcius(temp):
    for each_temp in temp:
        each_temp = (int(each_temp)- 32) * (5/9)    #converting to int type as it gives an error for float
        new_temp = round(each_temp)
        temp_celcius.append(f'{new_temp} °C')
    return temp_celcius
celcius(temp)

temp_celcius = temp_celcius[0:10]

In [7]:
#Extracting the summary description
summary = soup.find_all('p', class_ = 'DailyContent--narrative--3AcXd')

summary_list = []
for summary in summary:
    e = summary.text              
    summary_list.append(e)

summary_list = summary_list[0:10]

In [8]:
#creating the dataf

df = pd.DataFrame({'Date':date,'Day':type_of_day,'Temp':temp_celcius,'summary':summary_list}) 
print(df)

#df.to_csv('Weather_forecast_Hard.csv', index=False) #write the df to a csv file

     Date    Day   Temp                                            summary
0  Thu 10    Day  16 °C  Abundant sunshine. High around 60F. Winds W at...
1  Thu 10  Night  10 °C    A few clouds. Low 49F. Winds W at 10 to 20 mph.
2  Fri 11    Day  17 °C  Intervals of clouds and sunshine. High 62F. Wi...
3  Fri 11  Night  12 °C  A few clouds from time to time. Low 54F. Winds...
4  Sat 12    Day  18 °C  Partly cloudy skies. High 64F. Winds WSW at 10...
5  Sat 12  Night  12 °C  Partly cloudy during the evening followed by c...
6  Sun 13    Day  17 °C  Cloudy skies early, followed by partial cleari...
7  Sun 13  Night  13 °C  Cloudy skies. Low around 55F. Winds WSW at 10 ...
8  Mon 14    Day  17 °C  Cloudy early with partial sunshine expected la...
9  Mon 14  Night  11 °C  Partly cloudy skies. Low 52F. Winds WNW at 10 ...
